In [1]:
# =========================
# TyBALT-style VAE for Chimera RNA-seq (single-run script)
# - Input:  /content/rna_seq_raw.csv  (rows=patient_id like 3A_001, cols=genes)
# - Output: /content/chimera_rna_tybalt_latent.tsv
#           /content/models/encoder_onehidden_vae.hdf5
# =========================

# ---- (0) Install deps (Colab usually already has these; safe to run)
!pip -q install --upgrade "tensorflow>=2.12" pandas numpy scikit-learn matplotlib seaborn tqdm

import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Lambda, Layer, Activation, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import Callback

# -------------------------
# (1) Reproducibility
# -------------------------
np.random.seed(123)
tf.random.set_seed(123)

# -------------------------
# (2) Load Chimera RNA-seq
# -------------------------
RNA_CSV = "/content/rna_seq_raw.csv"  # <-- may have to change

rnaseq_df = pd.read_csv(RNA_CSV, index_col=0)
rnaseq_df.index = rnaseq_df.index.astype(str)
rnaseq_df = rnaseq_df.apply(pd.to_numeric, errors="coerce").fillna(0.0)

print("Raw Chimera RNA shape:", rnaseq_df.shape)
print("Index example:", rnaseq_df.index[:5].tolist())
print("Gene example:", rnaseq_df.columns[:5].tolist())

# -------------------------
# (3) Preprocess (match TyBALT "scaled_zeroone" expectation)
#     - log1p(counts)
#     - z-score per gene (across samples)
#     - min-max to [0,1] per gene (for sigmoid decoder)
# -------------------------
X = rnaseq_df.values.astype(np.float32)
#X = np.log1p(X) #i think chimera has already been logged, it range from 4 to 16

# z-score per gene: fit on samples => standardize columns
Xz = StandardScaler(with_mean=True, with_std=True).fit_transform(X)

# minmax per gene to [0,1]
X01 = MinMaxScaler(feature_range=(0.0, 1.0)).fit_transform(Xz)

rnaseq_scaled_df = pd.DataFrame(X01, index=rnaseq_df.index, columns=rnaseq_df.columns)
print("Scaled (log1p + z + minmax) shape:", rnaseq_scaled_df.shape)
print("Value range:", float(rnaseq_scaled_df.values.min()), float(rnaseq_scaled_df.values.max()))

# -------------------------
# (4) Train/test split
# -------------------------
test_set_percent = 0.1
rnaseq_test_df = rnaseq_scaled_df.sample(frac=test_set_percent, random_state=123)
rnaseq_train_df = rnaseq_scaled_df.drop(rnaseq_test_df.index)

print("Train shape:", rnaseq_train_df.shape, "Test shape:", rnaseq_test_df.shape)

# -------------------------
# (5) Warm-up beta callback (like notebook)
# -------------------------
class WarmUpCallback(Callback):
    """
    Gradually increases beta (KLD weight) during training.
    """
    def __init__(self, beta, kappa=1.0):
        super().__init__()
        self.beta = beta
        self.kappa = kappa

    def on_epoch_end(self, epoch, logs=None):
        # Increase beta by kappa each epoch until it reaches 1.0
        new_beta = min(K.get_value(self.beta) + self.kappa, 1.0)
        K.set_value(self.beta, new_beta)

# -------------------------
# (6) VAE components (TyBALT-style)
# -------------------------
# Hyperparams (keep close to TyBALT defaults)
original_dim = rnaseq_train_df.shape[1]
latent_dim = 100
intermediate_dim = 100  # one hidden layer
learning_rate = 5e-4
epsilon_std = 1.0

beta = K.variable(0.0)   # warm-up starts from 0
kappa = 1e-2             # small warmup step per epoch (adjust if needed)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=K.shape(z_mean), mean=0.0, stddev=epsilon_std)
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Encoder
rnaseq_input = Input(shape=(original_dim,), name="rnaseq_input")

h = Dense(intermediate_dim, kernel_initializer="glorot_uniform")(rnaseq_input)
h = BatchNormalization()(h)
h = Activation("relu")(h)

z_mean = Dense(latent_dim, name="z_mean")(h)
z_log_var = Dense(latent_dim, name="z_log_var")(h)

z = Lambda(sampling, output_shape=(latent_dim,), name="z")([z_mean, z_log_var])

# Decoder (sigmoid -> expects [0,1] input)
decoder_h = Dense(intermediate_dim, activation="relu", name="decoder_h")
decoder_out = Dense(original_dim, activation="sigmoid", name="decoder_out")

h_decoded = decoder_h(z)
x_decoded = decoder_out(h_decoded)

# Custom loss layer (reconstruction + beta*KLD)
class CustomVariationalLayer(Layer):
    def __init__(self, beta, **kwargs):
        super().__init__(**kwargs)
        self.beta = beta

    def call(self, inputs):
        x_true, x_pred, z_mean_, z_log_var_ = inputs

        # Reconstruction loss (binary cross-entropy summed over genes)
        recon = tf.reduce_sum(tf.square(x_true - x_pred), axis=1)

        # KLD
        kld = -0.5 * tf.reduce_sum(1 + z_log_var_ - tf.square(z_mean_) - tf.exp(z_log_var_), axis=1)

        loss = tf.reduce_mean(recon + self.beta * kld)
        self.add_loss(loss)
        return x_pred

vae_layer = CustomVariationalLayer(beta, name="vae_loss")([rnaseq_input, x_decoded, z_mean, z_log_var])
vae = Model(rnaseq_input, vae_layer, name="tybalt_vae")

adam = optimizers.Adam(learning_rate=learning_rate)
vae.compile(optimizer=adam)  # loss handled via add_loss

vae.summary()

# -------------------------
# (7) Train
# -------------------------
EPOCHS = 50        # change here
BATCH_SIZE = 16

hist = vae.fit(
    rnaseq_train_df.values,
    None,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    shuffle=True,
    validation_data=(rnaseq_test_df.values, None),
    callbacks=[WarmUpCallback(beta=beta, kappa=kappa)],
    verbose=1
)

# -------------------------
# (8) Build encoder model & export latent embeddings
# -------------------------
encoder = Model(rnaseq_input, z_mean, name="encoder")
Z = encoder.predict(rnaseq_scaled_df.values, batch_size=32, verbose=0)

Zdf = pd.DataFrame(Z, index=rnaseq_scaled_df.index, columns=[f"z{i+1}" for i in range(Z.shape[1])])
Zdf.index.name = "patient_id"

out_dir = "/content/results"
model_dir = "/content/models"
os.makedirs(out_dir, exist_ok=True)
os.makedirs(model_dir, exist_ok=True)

latent_path = os.path.join(out_dir, "chimera_rna_tybalt_latent.tsv")
Zdf.to_csv(latent_path, sep="\t")

encoder_path = os.path.join(model_dir, "encoder_onehidden_vae.hdf5")
encoder.save(encoder_path)

print("\n✅ Saved:")
print("latent:", latent_path, "shape:", Zdf.shape)
print("encoder:", encoder_path)

# -------------------------
# (9) (Optional) quick sanity check: variance in latent dims
# -------------------------
var = Zdf.var(axis=0).sort_values(ascending=False)
print("\nTop-5 latent dims by variance:")
print(var.head(5))

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.plot(hist.history['loss'], label='train')
if 'val_loss' in hist.history:
    plt.plot(hist.history['val_loss'], label='val')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('VAE training loss')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Zdf: (163, 100)
Z = Zdf.values

# PCA → 2D
pca = PCA(n_components=2, random_state=42)
Z_pca = pca.fit_transform(Z)

pca_df = pd.DataFrame(
    Z_pca,
    index=Zdf.index,
    columns=["PC1", "PC2"]
)

# Plot
plt.figure(figsize=(6, 5))
plt.scatter(pca_df["PC1"], pca_df["PC2"], s=40, alpha=0.8)
plt.xlabel(f"PC1 ({pca.explained_variance_ratio_[0]*100:.1f}% var)")
plt.ylabel(f"PC2 ({pca.explained_variance_ratio_[1]*100:.1f}% var)")
plt.title("Chimera RNA Embedding (TyBALT VAE, PCA)")
plt.grid(True)
plt.tight_layout()
plt.show()